In [245]:
import os
import random
from dotenv import load_dotenv
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import pickle
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process

In [4]:
load_dotenv()

True

In [5]:
boardgames = pd.read_csv('../data/boardgames_extend.csv', index_col='id')

In [6]:
users = pd.read_csv('../data/users.csv')

In [7]:
ratings = pd.read_csv('../data/ratings_cleaned.csv')

In [8]:
with open('../models/knn_model_cosine.pickle', 'rb') as file:
    model = pickle.load(file)

with open('../models/nmf_50.pickle', 'rb') as file:
    nmf = pickle.load(file)

with open('../models/P_50', 'rb') as file:
    P = np.load(file)    

In [9]:
uri = os.getenv('LOCAL_POSTGRES_BOARDGAMEGEEKS_URI')

engine = create_engine(uri, echo=False)

In [10]:
def lookup_boardgame(ids):    
    '''
    converts boardgame ids into boardgame names
    '''
    return boardgames.loc[ids, 'name'].tolist()

def list_to_query(ids):
    ids_str = [str(i) for i in ids]
    ids_str = ','.join(ids_str)
    return ids_str

def lookup_boardgame_sql(ids):
    ids_str = list_to_query(ids)
    query = f'SELECT boardgamename FROM boardgames WHERE boardgameid IN ({ids_str});'
    request = pd.read_sql(query, engine)
    return request['boardgamename'].tolist()

In [11]:
def lookup_user_id(user_name):
    '''
    returns the user id of a user
    '''
    user_id = users[users['user_name']==user_name]['user_id'].tolist()[0]
    return user_id

def lookup_user_id_sql(user_name):
    '''
    returns the user id of a user
    '''
    query = f'''SELECT userid FROM users WHERE username = '{user_name}';'''
    user_id = pd.read_sql(query, engine )['userid'][0]
    return user_id

In [12]:
def create_user_ratings(user_name):
    '''
    returns a dataframe with rated boardgames for a specified user
    '''
    user_id = lookup_user_id(user_name)
    user = ratings[ratings['user_id']==user_id]
    return user

def create_user_ratings_sql(user_name):
    '''
    returns a dataframe with rated boardgames for a specified user
    '''
    user_id = lookup_user_id_sql(user_name)
    query = f'''SELECT boardgameid,rating, userid FROM ratings WHERE userid = {user_id};'''
    user = pd.read_sql(query, engine)
    return user

In [13]:
def create_user_vector(user_name):
    '''
    returns a 1d array of the ratings of one user
    unrated boardgames = 0
    '''
    user = create_user_ratings(user_name)
    vector_length = ratings['boardgame_id'].max()
    vector = np.repeat(0, vector_length+1)
    vector[user['boardgame_id']] = user['ratings']
    return vector

def create_user_vector_sql(user_name):
    '''
    returns a 1d array of the ratings of one user
    unrated boardgames = 0
    '''
    user = create_user_ratings_sql(user_name)
    query = '''SELECT MAX(boardgameid) FROM boardgames;'''
    vector_length = pd.read_sql(query, engine)['max'][0]
    vector = np.repeat(0, vector_length+1)
    vector[user['boardgameid']] = user['rating']
    return vector

In [14]:
def values_to_list(df, column_name):
    categories = []
    for i in df[df[column_name].notna()].iterrows():
        categories = categories + i[1][column_name].split(', ')
    categories = list(dict.fromkeys(categories))
    categories.sort()
    return categories

def values_to_list_sql(column_name):
    categories = []
    query = f'''SELECT {column_name} FROM boardgames;'''
    df = pd.read_sql(query, engine)
    for i in df[df[column_name].notna()].iterrows():
        categories = categories + i[1][column_name].split(', ')
    categories = list(dict.fromkeys(categories))
    categories.sort()
    return categories

In [15]:
def user_rated_boardgames(user_name):
    query = f'''
        SELECT boardgames.boardgameid, boardgames.boardgamename, ratings.rating FROM boardgames
        JOIN ratings ON ratings.boardgameid = boardgames.boardgameid
        JOIN users ON users.userid = ratings.userid
        WHERE users.username = '{user_name}'
        ORDER BY ratings.rating DESC;
        '''
    df = pd.read_sql(query, engine)
    return df['boardgameid'].tolist(), df['boardgamename'].tolist(), df['rating'].tolist()

In [16]:
def neighbor_recommender(user_name):
    '''
    returns a list of boardgame recommendations
    '''
    user_ratings = create_user_ratings(user_name)
    user_vector = create_user_vector(user_name)
    #create neigbors of user
    distances, neighbor_ids = model.kneighbors([user_vector], n_neighbors=10)
    print(neighbor_ids)
    print(distances)
    neighbor_filter = ratings['user_id'].isin(neighbor_ids[0][1:])
    #create mean retings of games, rated by the neighbors
    neighbor_ratings = ratings[neighbor_filter].groupby('boardgame_id').mean()
    #sort rated games by mean rating
    neighbor_top = neighbor_ratings['ratings'].sort_values(ascending=False)
    #remove games which user rated already
    played_filter = ~neighbor_top.index.isin(user_ratings['boardgame_id'])
    recommend_ids = neighbor_top[played_filter].index
    return boardgames.loc[recommend_ids]['name'].tolist()[:20]

def neighbor_recommender_sql(user_name):
    '''
    returns a list of boardgame recommendations
    '''
    user_ratings = create_user_ratings_sql(user_name)
    user_vector = create_user_vector_sql(user_name)
    distances, neighbor_ids = model.kneighbors([user_vector], n_neighbors=10)
    neighbor_ids = list_to_query(neighbor_ids[0])
    boardgame_ids, boardgame_names, ratings = user_rated_boardgames(user_name)
    boardgame_ids = list_to_query(boardgame_ids)
    query = f'''
        SELECT boardgames.boardgamename, AVG(ratings.rating) FROM ratings 
        JOIN boardgames ON boardgames.boardgameid = ratings.boardgameid
        WHERE ratings.userid IN({neighbor_ids}) AND ratings.boardgameid NOT IN ({boardgame_ids})
        GROUP BY boardgames.boardgamename
        ORDER BY avg DESC
        LIMIT 20;
        '''
    return pd.read_sql(query, engine)['boardgamename'].tolist()


In [57]:
def knn_nmf_recommender(user_name):
    user_vector = create_user_vector(user_name)
    knn = NearestNeighbors(metric='cosine')
    knn.fit(P)
    vector_transformed = nmf.transform([user_vector])
    vector_transformed = vector_transformed.T.reshape(50,)
    distances, neighbor_ids = knn.kneighbors([vector_transformed], n_neighbors=20)
    print(neighbor_ids)
    user_ratings = create_user_ratings(user_name)
    played_by_user = user_ratings['boardgame_id']
    neighbor_ratings = ratings[ratings['user_id'].isin(neighbor_ids[0][1:])]
    neighbor_ratings = neighbor_ratings[(~neighbor_ratings['boardgame_id'].isin(played_by_user))]
    value_counts = pd.DataFrame(neighbor_ratings['boardgame_id'].value_counts())
    value_counts.columns = ['count']
    frequently_played = value_counts[value_counts['count']>=5].index
    recommendations = neighbor_ratings[neighbor_ratings['boardgame_id'].isin(frequently_played)].groupby('boardgame_id').mean().sort_values('ratings', ascending=False).head(15).index
    return boardgames.loc[recommendations]['name']

def knn_nmf_recommender_sql(user_name):
        user_vector = create_user_vector_sql(user_name)
        knn = NearestNeighbors(metric='cosine')
        knn.fit(P)
        vector_transformed = nmf.transform([user_vector])
        vector_transformed = vector_transformed.T.reshape(50,)
        distances, neighbor_ids = knn.kneighbors([vector_transformed], n_neighbors=20)
        print(neighbor_ids)
        neighbor_ids = list_to_query(neighbor_ids[0])
        user_ratings = create_user_ratings_sql(user_name)
        boardgame_ids, boardgame_names, ratings = user_rated_boardgames(user_name)
        boardgame_ids = list_to_query(boardgame_ids)
        query = f'''
                SELECT COUNT(ratings.boardgameid), boardgames.boardgamename, AVG(ratings.rating) FROM ratings 
                JOIN boardgames ON boardgames.boardgameid = ratings.boardgameid
                WHERE ratings.userid IN({neighbor_ids}) AND ratings.boardgameid NOT IN ({boardgame_ids}) 
                GROUP BY boardgames.boardgamename
                HAVING count(*)>4
                ORDER BY avg DESC
                LIMIT 15
                '''
        return pd.read_sql(query, engine)

In [149]:
def nmf_recommender(user_name):
    user_vector = create_user_vector(user_name)
    user_ratings = create_user_ratings(user_name)
    P = nmf.transform([user_vector])
    Q = nmf.components_
    predictions = np.dot(P, Q)
    #prediction include ids from 0 to length of user_vector
    #pseudo ids for constructing a dataframe
    pseudo_ids = list(range(0,len(user_vector)))
    df = pd.DataFrame(predictions, columns=pseudo_ids)
    recommendations_all = df.T.sort_values(0, ascending=False)
    recommendations_all = recommendations_all.reset_index()
    recommendations_all.columns = ['pseudo_id', 'pred_rating']
    #merging with boardgames dataframe keeps only existing boardgameids
    boardgames_merge = boardgames[['name']].reset_index()
    prediction_df = pd.merge(boardgames_merge[['id','name']], recommendations_all, left_on='id', right_on='pseudo_id', how='left')
    prediction_df = prediction_df.set_index('id')
    played_by_user = user_ratings['boardgame_id']
    played_filter = ~prediction_df.index.isin(played_by_user)
    return prediction_df.loc[played_filter].sort_values('pred_rating', ascending=False).head(15)
    #return prediction_df

def nmf_recommender_sql(user_name):
    user_vector = create_user_vector_sql(user_name)
    user_ratings = create_user_ratings_sql(user_name)
    P = nmf.transform([user_vector])
    Q = nmf.components_
    predictions = np.dot(P, Q)    
    #prediction include ids from 0 to length of user_vector
    #pseudo ids for constructing a dataframe
    pseudo_ids = list(range(0,len(user_vector)))
    df = pd.DataFrame(predictions, columns=pseudo_ids)
    #sort values: existing ids will be first, non-existing ids last (due to 0 value)
    recommendations_all = df.T.sort_values(0, ascending=False)
    recommendations_all = recommendations_all.reset_index()
    recommendations_all.columns = ['pseudo_id', 'pred_rating']
    boardgame_ids, boardgame_names, ratings = user_rated_boardgames(user_name)
    #filter already played ids from recommendations_all
    recommendations_all = recommendations_all[~recommendations_all['pseudo_id'].isin(boardgame_ids)]
    recommendation_ids = recommendations_all.head(15)['pseudo_id']
    recommendation_ids = list_to_query(recommendation_ids)
    print(recommendation_ids)
    query = f'''
            SELECT boardgameid, boardgamename FROM boardgames
            WHERE boardgameid IN({recommendation_ids})
            '''
    recommendations = pd.read_sql(query, engine)
    return recommendations

In [176]:
def values_to_list(df, column_name):
    categories = []
    for i in df[df[column_name].notna()].iterrows():
        categories = categories + i[1][column_name].split(', ')
    categories = list(dict.fromkeys(categories))
    categories.sort()
    return categories

def ohe_user_boardgames(user_name, column, weight=False):
    '''
    returns a one-hot-encoded matrix of parameters in column of games played by user
    if weight = True, the encoding gets weighted by the rating
    '''
    games_ohe={}
    user_id = users[users['user_name']==user_name]['user_id'].tolist()[0]
    user_ratings = ratings[ratings['user_id']==user_id].set_index('boardgame_id')
    user_boardgames = boardgames.loc[user_ratings.index]
    user_boardgames = user_boardgames[user_boardgames[column].notna()]
    user_categories = values_to_list(user_boardgames, column)    
    for i in user_boardgames.iterrows():
        game_vector = [0]*len(user_categories)
        for c in i[1][column].split(', '):
            index = user_categories.index(c)
            if weight == True:
                game_vector[index]=1 * user_ratings.loc[i[0]]['ratings']
            else: 
                game_vector[index]=1
        games_ohe[i[0]] = game_vector
    df = pd.DataFrame(games_ohe)
    df = df.transpose()
    df.columns = user_categories
    if 'Deck' in user_categories:
        df['Deck, Bag and Pool Building'] = df['Deck']
        df.drop(columns=['Deck', 'Bag', 'and Pool Building'], inplace=True)
    if 'I Cut' in user_categories:
        df['I Cut, You Choose'] = df['I Cut']
        df.drop(columns=['I Cut', 'You Choose'], inplace=True)
    return df

def ohe_user_boardgames_sql(user_name, column, weight=False):
    games_ohe={}
    boardgame_ids, boardgame_names, ratings = user_rated_boardgames(user_name)
    boardgame_ids = list_to_query(boardgame_ids)
    query = f'''
            SELECT boardgameid, boardgamename, {column} FROM boardgames
            WHERE boardgameid IN({boardgame_ids});
            '''
    user_boardgames = pd.read_sql(query, engine)
    user_boardgames = user_boardgames[user_boardgames[column].notna()]
    user_categories = values_to_list(user_boardgames, column)    
    for i in user_boardgames.iterrows():
        game_vector = [0]*len(user_categories)
        for c in i[1][column].split(', '):
            index = user_categories.index(c)
            if weight == True:
                game_vector[index]=1 * user_ratings.loc[i[0]]['ratings']
            else: 
                game_vector[index]=1
        games_ohe[i[0]] = game_vector
    df = pd.DataFrame(games_ohe)
    df = df.transpose()
    df.columns = user_categories
    if 'Deck' in user_categories:
        df['Deck, Bag and Pool Building'] = df['Deck']
        df.drop(columns=['Deck', 'Bag', 'and Pool Building'], inplace=True)
    if 'I Cut' in user_categories:
        df['I Cut, You Choose'] = df['I Cut']
        df.drop(columns=['I Cut', 'You Choose'], inplace=True)
    return df

In [180]:
ohe_user_boardgames_sql('Ser0', 'machanics', weight=False).sum().sort_values(ascending=False).index.tolist()[:5]

['Hand Management',
 'Dice Rolling',
 'Variable Player Powers',
 'Card Drafting',
 'Solo / Solitaire Game']

In [226]:
def lookup_boardgamename(search_query):
    """
    returns boardgamename after comparing user input with boardgamenames
    """
    query = 'SELECT boardgameid, boardgamename FROM boardgames'
    result = pd.read_sql(query, engine)
    match = process.extractOne(search_query, result['boardgamename'])
    boardgame_id = result[result['boardgamename']==match[0]]['boardgameid']
    # [(title, score, movieId), ...]
    return match, boardgame_id

In [236]:
def random_recommender(N):
    '''
    returns a list of N random boardgames
    '''
    rec_ids = []
    for n in range(0,N):
        rec_ids.append(random.choice(boardgames.index))
        recommendations = lookup_boardgame(rec_ids)
    return recommendations

In [250]:
query = '''
        SELECT boardgameid FROM boardgames
        '''
boardgame_ids = pd.read_sql(query, engine)['boardgameid']
#boardgame_ids
rec_ids = []
for n in range(0,5):
    rec_ids.append(random.choice(boardgame_ids))
    recommendations = lookup_boardgame_sql(rec_ids)

recommendations

['Panzer Grenadier',
 'Monopoly Gamer Power Pack: Rosalina',
 'Gangsta!',
 'Uncharted: The Board Game – Nathan Drake & Victor Sullivan',
 'Odisea Espacial']

In [239]:
boardgames.columns.boardgameid

AttributeError: 'Index' object has no attribute 'boardgameid'

In [184]:
boardgames.loc[119]

Unnamed: 0                                                    782
rank                                                         1376
name                                                     Kingdoms
links                                     /boardgame/119/kingdoms
num_voters                                                   4675
categories                   Abstract Strategy, Fantasy, Medieval
mechanics                                          Tile Placement
family                         Series: Fantasy Flight Silver Line
expansions                                   Kingdoms: The Wizard
integrations                                                  NaN
designers                                           Reiner Knizia
publishers      Hans im Glück, ADC Blackfire Entertainment, Ar...
Name: 119, dtype: object

## QUERIES
- SELECT * FROM users WHERE username = 'Ser0';
-     SELECT boardgames.boardgamename FROM boardgames JOIN ratings ON ratings.boardgameid = boardgames.boardgameid JOIN users ON users.userid = ratings.userid WHERE users.username = 'Ser0';

In [ ]:
pd.read_sql('SELECT * FROM boardgames LIMIT 10', engine)

In [ ]:
pd.read_sql('SELECT * FROM ratings LIMIT 10', engine)

In [89]:
boardgames.loc[183394]

Unnamed: 0                                                     53
rank                                                           24
name                                Viticulture Essential Edition
links             /boardgame/183394/viticulture-essential-edition
num_voters                                                  31378
categories                                      Economic, Farming
mechanics       Contracts, Hand Management, Solo / Solitaire G...
family          Components: Control Boards, Country: Italy, Di...
expansions      Tuscany: Special Worker Promo Cards, Viticultu...
integrations                                                  NaN
designers                             Jamey Stegmaier, Alan Stone
publishers      Stonemaier Games, Angry Lion Games, Arclight, ...
Name: 183394, dtype: object

In [ ]:
        query = f'''
                SELECT boardgames.boardgamename, AVG(ratings.rating), COUNT(boardgames.boardgamename) FROM ratings 
                JOIN boardgames ON boardgames.boardgameid = ratings.boardgameid
                WHERE ratings.userid IN({neighbor_ids}) AND ratings.boardgameid NOT IN ({boardgame_ids}) 
                GROUP BY boardgames.boardgamename
                HAVING count(*)>1
                ORDER BY avg DESC
                LIMIT 15;
                '''